In [1]:
# При запуске ноутбука рабочей директорией ставится не папка проекта, а папка examples, а в ней нет src, 
# соответственно импорты не работают. Эта строчка фиксит такое поведение для тестирования, у пользователей будет
# свой путь установки, в котором этой проблемы не будет. 
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd

from src.eventstream.eventstream import Eventstream
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.graph.p_graph import PGraph, EventsNode

raw_data = pd.read_csv('simple-onlineshop.csv')

raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

source = Eventstream(
    raw_data=raw_data,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

graph = PGraph(source_stream=source)

In [3]:
# Add simple group
from src.data_processors_lib.simple_processors import SimpleGroup, SimpleGroupParams

cart_events = EventsNode(
    SimpleGroup(params=SimpleGroupParams(**{
        "event_name": "add_to_cart",
        "filter": lambda df, schema: df[schema.event_name].isin(["cart"]),
    }))
)
graph.add_node(
    node=cart_events,
    parents=[graph.root]
)

In [4]:
from src.data_processors_lib.rete import StartEndEvents, StartEndEventsParams

start_end_events = EventsNode(
    StartEndEvents(params=StartEndEventsParams(**{}))
)

graph.add_node(
    node=start_end_events,
    parents=[cart_events]
)

result = graph.combine(
    node=start_end_events
)
result.to_dataframe().head(n=20)


event_id   event_type  event_index  \
0   8a99cd48-f902-4cf8-8271-52f38253a0ea        start            0   
2   118d6a43-14be-439a-8d31-f0935519bbd0          raw            2   
3   b7edda36-bf30-4780-a64d-02c12cac1e14  group_alias            3   
5   0c5176ad-208d-4733-8888-8b9b381438f1          raw            5   
6   a2f754a3-9eff-4e53-b6ac-46c88185be41        start            6   
8   ad5f603d-0b2e-4b0d-bc52-8f4fd41f564c  group_alias            8   
10  e5bde0d6-35df-4ad8-8851-050f1f040f82          raw           10   
11  c567c8a3-8a9e-47f2-b341-68cd8140990c        start           11   
12  d822f1bf-0979-4d9e-92be-57fc90a64aa2        start           12   
15  d09836b0-0e96-467e-9c65-66dd1d927214          raw           15   
16  56f62042-1ab1-4f43-9cc8-5c7dfabb03d8          raw           16   
17  f127d068-808c-4e64-b76d-0c2fc463a2a7          raw           17   
18  73aa8a7a-2081-4d52-a77c-65d6dd8dffbc          raw           18   
19  6df1dccd-d1a8-4d10-8f7f-5d8156dfc2a8          raw           19   
20  2efa6053-523a-40ca-bcd0-3d42fec9d680          raw           20   
21  adcc0aec-e3c6-4f7d-97b8-76d70ffe6332          raw           21   
22  aaf286b4-7d21-4660-a65a-3ff0ef3904b2          raw           22   
23  f5949f2b-f8d7-4512-8607-80a52f3b0ea6          raw           23   
24  cff5996c-f945-45f1-95b1-8757ea173ae5          raw           24   
25  bdabcba9-d219-4705-be20-390fef3717f5          raw           25   

         event_name            event_timestamp      user_id  
0             start 2019-11-01 17:59:13.273932  219483890.0  
2          product1 2019-11-01 17:59:28.459271  219483890.0  
3       add_to_cart 2019-11-01 17:59:29.502214  219483890.0  
5           catalog 2019-11-01 17:59:32.557029  219483890.0  
6             start 2019-11-01 21:38:19.283663  964964743.0  
8       add_to_cart 2019-11-01 21:38:36.761221  964964743.0  
10  delivery_choice 2019-11-01 21:38:37.564693  964964743.0  
11            start 2019-11-01 22:28:54.791683  629881394.0  
12            start 2019-11-01 22:28:54.791683  629881395.0  
15          catalog 2019-11-01 22:29:01.049513  629881394.0  
16          catalog 2019-11-01 22:29:01.049513  629881395.0  
17          catalog 2019-11-01 22:29:32.322458  629881394.0  
18          catalog 2019-11-01 22:29:32.322458  629881395.0  
19          catalog 2019-11-01 22:30:09.450839  629881394.0  
20          catalog 2019-11-01 22:31:05.565762  629881394.0  
21             main 2019-11-01 22:31:08.333560  629881394.0  
22          catalog 2019-11-01 22:31:09.010626  629881394.0  
23         product1 2019-11-01 22:31:10.416231  629881394.0  
24          catalog 2019-11-01 22:31:43.019527  629881394.0  
25          catalog 2019-11-01 22:32:01.596163  629881394.0